# Red Tide Detection in South Florida

Detecting Red Tide events in southwest Florida based on known events from 25 October 2024

In [1]:
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# ee.Authenticate()
ee.Initialize()

In [62]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

def cloudMask(image):
    clouds = ee.Algorithms.Landsat.simpleCloudScore(image).select(['cloud'])
    return image.updateMask(clouds.lt(10))

In [82]:
# Load datasets
LANDSAT = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
              .map(cloudMask) \
             # .filterDate('2024-01-01', '2024-10-26') \

             # .filterBounds(ee.Geometry.Point(26.74, -82.3))

SENTINEL = ee.ImageCollection("COPERNICUS/S3/OLCI") \
             .filterDate('2024-10-17', '2024-10-31') \
             # .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

# aquaMODIS = ee.ImageCollection("NASA/OCEANDATA/MODIS-Aqua/L3SMI") \
#              .filterDate('2024-10-20', '2024-10-31') \
#              .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

VIIRS = ee.ImageCollection("NASA/VIIRS/002/VNP09GA") \
             .filterDate('2024-10-17', '2024-10-31') \
             # .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

GCOM_C = ee.ImageCollection("JAXA/GCOM-C/L3/OCEAN/CHLA/V3") \
             .filterDate('2024-10-17', '2024-10-31') \
             # .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D'))

In [83]:
# Create a basemap
Map = geemap.Map(center=[27, -81], zoom=8.5)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

In [84]:
meanLANDSAT = LANDSAT.reduce(ee.Reducer.mean())
vizParamsLANDSAT_mean = {'bands': ['B4_mean', 'B3_mean', 'B2_mean'], 'min': 5000, 'max': 25000, 'gamma': 1.3}
vizParamsLANDSAT = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 65000, 'gamma': 1.3}
Map.addLayer(meanLANDSAT, vizParamsLANDSAT_mean, 'Landsat 8 Mean')

In [85]:
Map

Map(center=[27, -81], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [75]:
LANDSAT